# Part 1: Animal Density Demo

In this set of tutorials, we will walk through an example of how you might take animal density data from [MGEL Density Models: US East Coast](https://seamap.env.duke.edu/models/Duke/EC/) and extract density information from specific areas. 

We'll be combining the density data with polygon shapefiles from the BOEM offshore wind lease areas (Find this data at [BOEM Renewable Energy GIS Data site](https://www.boem.gov/renewable-energy/mapping-and-data/renewable-energy-gis-data)). We want to basically be able to compute the average density in the vicinity of the lease areas - something that can help understand how many animals may be affected by activities related to the installation, operation, or decommissioning of offshore wind turbines. 

To show you where we're headed, here's an example image that displays part of an animal density map. We have a lease area outline, along with a 10km buffer around that lease area. This set of tutorials will show you how to pull all of these pieces together and then get the average density within the buffere area.

![example-buffer-plot](../images/density_buffer_plots/example_buffer_plot.png)


In [1]:
# Imports
import rasterio
import os
import glob
from collections import Counter
import numpy as np

## Setting up the density data

The purpose of "Part 1" (this notebook) is to prepare the density data so that we can work with it more easily in subsequent steps. 

You can download the density data into one big zip file from [this web page (MGEL/Duke)](https://seamap.env.duke.edu/models/Duke/EC/). Note that for this tutorial, I'm assuming you grab the zip file that includes all of the species rather than individually downloading them. It is technically the same thing but I use the specific organizational structure that you get from the big zip download, so it'll be easier to follow if you keep it like that. 

Since we're working with large files, I'm not allowing the data to be stored on the repository. So you'll need to make sure to gather it and put it into your own folders as described below. 

Unzip your zip file and put them contents of that folder into a folder called "data/density", like this:

    |--data
        |-- density
            |-- Atlantic_spotted_dolphin_v9.1
            |-- Atlantic_white_sided_dolphin_v4.1
            |-- etc...




In [10]:
# First, we'll find all of the data files
density_folder = "../data/density"
species_folders = [path for path in glob.glob(os.path.join(density_folder, '*')) if os.path.isdir(path)]

# Use glob to find all .img files in all subdirectories
all_img_files = glob.glob(f'{density_folder}/**/*.img', recursive=True)

# Filter for files that contain the word "density"
density_img_files = [file for file in all_img_files if 'density' in os.path.basename(file)]

# Print out the first 5 files just to see what it looks like:
density_img_files[0:5]

['../data/density/Dwarf_and_pygmy_sperm_whales_v5.1/Rasters/UNKO_v5.1_density.img',
 '../data/density/Humpback_whale_v11.1/Rasters/2002-2019/HUWH_v11.1_2002_2019_density_month10.img',
 '../data/density/Humpback_whale_v11.1/Rasters/2002-2019/HUWH_v11.1_2002_2019_density_month04.img',
 '../data/density/Humpback_whale_v11.1/Rasters/2002-2019/HUWH_v11.1_2002_2019_density_month05.img',
 '../data/density/Humpback_whale_v11.1/Rasters/2002-2019/HUWH_v11.1_2002_2019_density_month11.img']

In [21]:
# Get species name from the path
species_name = [filepath.split('/')[3].split('_v')[0] for filepath in density_img_files]

# 
count_of_species = Counter(species_name)

# Generate a list of unique species for which we have density data
np.unique(species_name)

array(['Atlantic_spotted_dolphin', 'Atlantic_white_sided_dolphin',
       'Blue_whale', 'Clymene_dolphin', 'Common_bottlenose_dolphin',
       'Common_minke_whale', 'Cuviers_beaked_whale',
       'Dwarf_and_pygmy_sperm_whales', 'False_killer_whale', 'Fin_whale',
       'Frasers_dolphin', 'Harbor_porpoise', 'Humpback_whale',
       'Killer_whale', 'Melon_headed_whale', 'Mesoplodont_beaked_whales',
       'North_Atlantic_right_whale', 'Northern_bottlenose_whale',
       'Pantropical_spotted_dolphin', 'Pilot_whales',
       'Pygmy_killer_whale', 'Rissos_dolphin', 'Rough_toothed_dolphin',
       'Seals', 'Sei_whale', 'Short_beaked_common_dolphin', 'Sperm_whale',
       'Spinner_dolphin', 'Striped_dolphin', 'Unidentified_beaked_whales',
       'White_beaked_dolphin'], dtype='<U28')

### Density data - special cases

Most species have a density file for each month, but not all. For consistency, however, and to make plotting easier later on, we want to have a file for each month. 

If you look at "count_of_species" printout below, you can see that several species have only 1 file, and are simple annual averages. In that case we will make a separate file for each month that's a copy of the annual file (this is for plotting purposes). 

Two species have 36 files: humpback whale and North Atlantic right whale. These are special cases that have explanations in their respective readme files. For Humpback whales, the authors recommend using the 2009-2019 files. For NARW, they recommend using 2010-2019.


In [32]:
# List out a few of the species counts as an example
list(count_of_species.items())[:5]

[('Dwarf_and_pygmy_sperm_whales', 1),
 ('Humpback_whale', 36),
 ('Spinner_dolphin', 1),
 ('Sperm_whale', 12),
 ('Striped_dolphin', 1)]

## Define Img-to-geotiff function

We are going to be reading in all of the img files, and saving them as geotifs into the same directory, with consistent naming. Since we're doing the img-to-geotif conversion many times, we can just define a function that we can call over and over again. 

In [23]:
def img_to_geotiff(input_file_path, output_file_path):
    """
    Convert ERDAS Imagine files to geotiff

    Args:
        input_file_path (str): Path to *.img file
        output_file_path (str): Path to *.tif file

    Returns: None

    """
    with rasterio.open(input_file_path) as src:
        # Read the data
        data = src.read()
        # Copy the metadata
        meta = src.meta.copy()
        # Update the metadata for GeoTIFF
        meta.update(driver='GTiff')
        # Write to a new GeoTIFF file
        with rasterio.open(output_file_path, 'w', **meta) as dst:
            dst.write(data)

## Convert and organize the density files

This is where all the work happens - below we ahve a for-loop, where we're looping through every density file, renaming it so that it is easy to parse species and month later on, and re-saving all of those files as geotiffs into the same directory. 

In [35]:
for density_file, spp_name in zip(density_img_files, species_name):

    # There are 12 months of density data available
    if count_of_species.get(spp_name)==12:
        month_number = density_file.split('month')[1].split('.img')[0]
        out_file = f"../data/density_geotiffs/{spp_name}.month{month_number}.tif"
        img_to_geotiff(density_file, out_file)

    # There's only an annual average
    elif count_of_species.get(spp_name)==1:
        for month_num_val in np.arange(1,13):
            out_file = f"../data/density_geotiffs/{spp_name}.month{month_num_val:02}.tif"
            img_to_geotiff(density_file, out_file)

    # Humpback whale - special case
    elif spp_name == 'Humpback_whale':
        if "2009_2019" in density_file:
            month_number = density_file.split('month')[1].split('.img')[0]
            out_file = f"../data/density_geotiffs/{spp_name}.month{month_number}.tif"
            img_to_geotiff(density_file, out_file)

    # NARW - special case
    elif spp_name == 'North_Atlantic_right_whale':
        if "2010-2019" in density_file:
            month_number = density_file.split('month')[1].split('.img')[0]
            out_file = f"../data/density_geotiffs/{spp_name}.month{month_number}.tif"
            img_to_geotiff(density_file, out_file)

    else:
        print('Error! This file was not handled: ' + density_file)


## End of Part 1

That's it for Part 1! We have now prepared all of the density files for the next step, where we'll be calculating some specific statistics. 